In [1]:
{-# LANGUAGE TypeOperators, ParallelListComp #-}
:opt no-lint

In [2]:
import Math.LinearMap.Category
import Prelude ()
import Control.Category.Constrained.Prelude
type ℝ = Double

In [3]:
import System.Random
import Control.Monad (replicateM)
randomPoints <- replicateM 100 $ randomRIO (0,10) :: IO [ℝ]
randomDeviations <- cycle <$> replicateM 165 `id` randomRIO (-0.1, 0.1) :: IO [ℝ]

In [4]:
mapM_ print (
 linearRegressionW euclideanNorm
    (\x -> lfun $ \(a,b) -> a*x + b)
 <$> [ [(0,1), (1,2), (2,3)]
     , [(-2,6), (0,2), (1,0)]
     , [(x, 14.3*x + 2.9 + δy) | x<-randomPoints
                               | δy<-randomDeviations]
     :: [(ℝ,ℝ)] ])

(1.0,1.0)
(-2.0,1.9999999999999996)
(14.300894549761438,2.895267024166003)

In [5]:
import Linear.V2 (V2(..))

mapM_ print (
 linearRegressionW euclideanNorm
    (\x -> lfun $ \((a,b),c) -> x<.>^(a$x) + b<.>^x + c)
 <$> [ [ (V2 x y, 3*x + 2*y + x*y + δy)
       | x<-randomPoints
       , y<-randomPoints
       | δy<-randomDeviations ]
     ] :: [((V2 ℝ+>V2 ℝ, V2 ℝ), ℝ)] )

((ex.<V2 (-5.791184475261406e11) 1.4110566378963125e12 ^+^ ey.<V2 (-1.44725154086525e12) 0.21776139905938763,V2 6.056172103839e12 0.0),-1.142963735399825e13)